In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import math
# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# **Data preprocessing**

In [ ]:
dt=pd.read_csv("/kaggle/input/digit-recognizer/train.csv")
dtest=pd.read_csv("/kaggle/input/digit-recognizer/test.csv")

In [ ]:
print(dt.head)
print("dg")

In [ ]:
Y=dt.iloc[:,0].values
X=dt.iloc[:,1:].values
dtest__=dtest.iloc[:,:].values
print("the number of samples "+ str(X.shape[0]))
print("the shape of trainig data "+ str(X.shape))

In [ ]:
#Encoding the target 
y_train_=np.zeros((10,42000))
for col in range (42000):
    val=Y[col]
    for row in range (10):
        if (val==row):
            y_train_[val,col]=1

In [ ]:
#reshaping data 
x_train_flatten=X.reshape(X.shape[0],-1).T
dtest_flatten=dtest__.reshape(dtest__.shape[0],-1).T


In [ ]:
#Standardize data 
X_train=x_train_flatten/255
X_test=dtest_flatten/255

# **Modeling**

In [ ]:
def initialize_parameters_deep(layer_dims):
    np.random.seed(1)
    parameters = {}
    L = len(layer_dims)
    for l in range(1, L):
        parameters['W' + str(l)] =np.random.randn(layer_dims[l],layer_dims[l-1])*0.01
        parameters['b' + str(l)] = np.zeros((layer_dims[l],1))
    return parameters

In [ ]:
def linear_forward(A, W, b):     
    Z = np.dot(W,A)+b
    assert(Z.shape == (W.shape[0], A.shape[1]))
    cache = (A, W, b)
    
    return Z, cache

In [ ]:
def relu(Z):
    A = np.maximum(0,Z)    
    cache = Z 
    return A, cache

def softmax(Z):
    A= np.exp(Z)/np.sum(np.exp(Z),axis=0)  
    cache=Z
    return A,cache   

In [ ]:
def linear_activation_forward(A_prev, W, b, activation):
    Z,linear_cache = linear_forward(A_prev, W, b)
    if activation == "relu": 
        A, activation_cache = relu(Z)
    elif activation == "softmax":
             
             A, activation_cache = softmax(Z) 
            
            
    assert (A.shape == (W.shape[0], A_prev.shape[1]))
    cache = (linear_cache, activation_cache)


    return A, cache

In [ ]:
def L_model_forward(X, parameters):
    caches = []
    A = X
    L = len(parameters) // 2   
    for l in range(1, L):
        A_prev = A 
        A, cache = linear_activation_forward(A_prev, parameters['W' + str(l)],parameters['b' + str(l)], activation = "relu")
        caches.append(cache)
  
    AL, cache = linear_activation_forward(A, parameters['W' + str(L)],parameters['b' + str(L)], activation = "softmax")
    caches.append(cache)
    
    #assert(AL.shape == (1,X.shape[1]))
            
    return AL, caches

In [ ]:
def compute_cost(AL, Y):
    m = Y.shape[1]
    #print(AL.shape)
    Y= np.reshape(Y,(Y.shape[0],Y.shape[1]))
    cost = (-1/m)*(np.multiply(Y,np.log(AL)))
    
    cost = np.squeeze(cost) 
    #assert(cost.shape == ())
    
    return cost


In [ ]:
def linear_backward(dZ, cache):
    A_prev, W, b = cache
    m = A_prev.shape[1]
    dW = 1/m*np.dot(dZ,A_prev.T)
    db = 1/m*np.sum(dZ,axis=1,keepdims=True)
    dA_prev = np.dot(W.T,dZ)
    
    assert (dA_prev.shape == A_prev.shape)
    assert (dW.shape == W.shape)
    assert (db.shape == b.shape)
    
    return dA_prev, dW, db

In [ ]:
def relu_backward(dA, cache):
    Z = cache
    dZ = np.array(dA, copy=True)
    dZ[Z <= 0] = 0
    assert (dZ.shape == Z.shape)
    return dZ
def softmax_backward(Z,cache):
    Z=cache
    length=10  
    dZ=np.zeros((29400,10))
    Z=np.transpose(Z)
    for row in range (0,29400):
            den=(np.sum(np.exp(Z[row,:])))*(np.sum(np.exp(Z[row,:])))
            for col in range (0,10):
                sums=0
                for j in range (0,10):
                    if (j!=col):
                        sums=sums+(math.exp(Z[row,j]))
                
                dZ[row,col]=(math.exp(Z[row,col])*sums)/den           
    dZ=np.transpose(dZ)
    Z=np.transpose(Z)

    assert (dZ.shape == Z.shape)
    return dZ

In [ ]:
def relu_prime(Z):
    Z[Z<=0] = 0
    Z[Z>0] = 1
    return Z

In [ ]:

def linear_activation_backward(dA, cache, activation):
   
    linear_cache, activation_cache = cache
    
    if activation == "relu":
        dZ = dA*relu_prime(activation_cache)
        dA_prev, dW, db = linear_backward(dZ,linear_cache)
        
    elif activation == "softmax":
        dZ = dA
        dA_prev, dW, db =linear_backward(dZ,linear_cache)
        
    
    return dA_prev, dW, db

In [ ]:
def L_model_backward(AL, Y, caches):
    grads = {}
    L = len(caches) # the number of layers
    m = AL.shape[1]
    Y = Y.reshape(AL.shape) # after this line, Y is the same shape as AL
    
    # Initializing the backpropagation
    
    dAL = AL-Y
   
    current_cache = caches[L-1]
    grads["dA" + str(L-1)], grads["dW" + str(L)], grads["db" + str(L)] = linear_activation_backward(dAL, caches[L-1], "softmax")
   
    
    for l in reversed(range(L-1)):
       
        current_cache = caches[l]
        dA_prev_temp, dW_temp, db_temp = linear_activation_backward(grads["dA" + str(l + 1)], current_cache, "relu")
        grads["dA" + str(l)] =dA_prev_temp
        grads["dW" + str(l + 1)] =dW_temp
        grads["db" + str(l + 1)] =db_temp
        

    return grads

In [ ]:
def update_parameters(parameters, grads, learning_rate):
    
    L = len(parameters) // 2 # number of layers in the neural network

    for l in range(L):
        parameters["W" + str(l+1)] =parameters["W" + str(l+1)]-learning_rate*(grads["dW" + str(l+1)])
        parameters["b" + str(l+1)] =parameters["b" + str(l+1)]-learning_rate*(grads["db" + str(l+1)])
    return parameters

In [ ]:
def mini_batch_gradient_descent(X,Y,layer_dims,mini_batch_size,epochs, learning_rate=0.0075):
  ############## Creating Mini Batches ##########################
    np.random.seed(1)
    m = X.shape[1]
    mini_batches = []
    
    #shuffling the data
    permutation = list(np.random.permutation(X.shape[1]))
    X_shuffled = X[:, permutation]
    Y_shuffled = Y[:, permutation]
    num_of_complete_batches = m // mini_batch_size
    for i in range(num_of_complete_batches):
        mini_batch_X = X_shuffled[:,i*mini_batch_size:(i+1)*mini_batch_size]
        mini_batch_Y = Y_shuffled[:,i*mini_batch_size:(i+1)*mini_batch_size]
        mini_batch = (mini_batch_X,mini_batch_Y)
        mini_batches.append(mini_batch)
        
    #if there is incomplete batch
    if m % mini_batch_size != 0:
       
        mini_batch_X=X_shuffled[:,num_of_complete_batches*mini_batch_size:num_of_complete_batches*mini_batch_size + (m - mini_batch_size*num_of_complete_batches)]
        mini_batch_Y=Y_shuffled[:,num_of_complete_batches*mini_batch_size:num_of_complete_batches*mini_batch_size + (m - mini_batch_size*num_of_complete_batches)]
        mini_batch = (mini_batch_X,mini_batch_Y)
        mini_batches.append(mini_batch)
    
    ############## Training Model Using Mini Batches ##########################
    #parameters_initialize
    costs=[]
    parameters = initialize_parameters_deep(layer_dims)
    for j in range(epochs):
        for mini_batch in mini_batches:
            x_batch,y_batch=mini_batch
            A_Last,caches = L_model_forward(x_batch,parameters)
            cost = np.sum(compute_cost(A_Last, y_batch))/A_Last.shape[0]
            grads = L_model_backward(A_Last, y_batch , caches)
            parameters = update_parameters(parameters, grads, learning_rate)
        if j%2 == 0:
            print(f'Iteration {j} : {cost}')
            costs.append(cost)
    return parameters,costs

# Training the model

In [ ]:
layers_dims = [784,100,40,10]

In [ ]:
parameters,costs =  mini_batch_gradient_descent(X_train,y_train_,layers_dims,mini_batch_size=9,epochs =60,learning_rate = 0.006 )

In [ ]:
plt.plot(costs)
plt.ylabel('Cost')
plt.xlabel('Iterations per 2 epochs')
plt.show()

# Make Predection 

In [ ]:
last_test,caches=L_model_forward(X_test,parameters)

In [ ]:
pred=np.argmax(last_test,axis=0)

# Submission

In [ ]:
df_submission = pd.read_csv('/kaggle/input/digit-recognizer/sample_submission.csv')
df_submission['Label'] = pred
df_submission.head()

In [ ]:
df_submission.to_csv('output.csv', index=False)